In [1]:
import sqlite3
import pandas as pd
import json

In [2]:
DATABASE = "./recruit.db"

In [3]:
conn = sqlite3.connect(DATABASE)

In [10]:
education = pd.read_sql('SELECT * FROM education', conn)
race = pd.read_sql('SELECT * FROM race', conn)
insurance_segment = pd.read_sql('select * FROM insurance_segment', conn)
customer = pd.read_sql('select * FROM customer', conn)

In [ ]:
test.groupby('economic_stability').sum()['cnt'].max()

In [ ]:
customerjson = json.loads(customer.to_json(orient='split'))

In [ ]:
def dictfetchall(cursor):
    "Return all rows from a cursor as a dict"
    columns = [col[0] for col in cursor.description]
    return [
        dict(zip(columns, row))
        for row in cursor.fetchall()
    ]


def query_db(query):

    result_dict = {}

    try:
        connection = sqlite3.connect(DATABASE)

        cursor = connection.cursor()
        cursor.execute(query)
        result_dict = dictfetchall(cursor)

    except sqlite3.OperationalError as e:
        print("Db operation error", e)
        result_dict["error"] = str(e)
    except:
        e = sys.exc_info()[0]
        print("An error occurred with the database", e)
        result_dict["error"] = str(e)
    else:
        cursor.close()
        connection.close()

    return result_dict

In [ ]:
connection = sqlite3.connect(DATABASE)
cursor = connection.cursor()
query = 'SELECT customer.economic_stability, insurance_segment.value, count(customer.economic_stability) as cnt \
            FROM customer \
            INNER JOIN insurance_segment ON customer.insurance_segment_id = insurance_segment.id \
            GROUP BY customer.economic_stability, insurance_segment.value'
cursor.execute(query)


In [13]:
insurance_segment

,id,value
0,1,Price Isn't Primary
1,2,Stretched Singles
2,3,Hands Full
3,4,The Nest is Empty
4,5,Country Middles
5,6,Solid Suburbans
6,7,Full House
7,8,Getting By


In [14]:
customer.head()

,id,race_code,education_id,home_owner,state,is_smoker,is_exerciser,has_insurance,income,travel_spending,sports_leisure_spending,economic_stability,insurance_segment_id,youtube_user_rank,facebook_user_rank,gender
0,1,B,1.0,,NE,,,,25,33,NaN,27,1,10,10,M
1,2,W,2.0,O,FL,,,,357,136,NaN,08,6,19,18,M
2,3,O,NaN,O,NE,,1,,25,18,5.0,13,1,19,19,M
3,4,O,2.0,,MO,,1,,51,527,NaN,23,6,14,16,M
4,5,O,NaN,,MO,,1,,51,527,NaN,23,6,14,16,F


In [20]:
data = dict()

for key in insurance_segment['value']:
    data[key] = []

query = 'SELECT customer.youtube_user_rank, insurance_segment.value\
            FROM customer \
            INNER JOIN insurance_segment ON customer.insurance_segment_id = insurance_segment.id'


connection = sqlite3.connect(DATABASE)

cursor = connection.cursor()
cursor.execute(query)

for row in cursor.fetchall():
    data[row[1]].append(row[0])

In [21]:
data

{"Price Isn't Primary": ['10',
  '19',
  '07',
  '11',
  '15',
  '19',
  '18',
  '19',
  '18',
  '17',
  '03',
  '16',
  '15',
  '08',
  '14',
  '15',
  '18',
  '18',
  '20',
  '17',
  '19',
  '17',
  '11',
  '16',
  '17',
  '15',
  '19',
  '20',
  '08',
  '19',
  '16',
  '06',
  '14',
  '18',
  '20',
  '17',
  '20',
  '13',
  '17',
  '16',
  '13',
  '20',
  '16',
  '18',
  '14',
  '19',
  '05',
  '08',
  '07',
  '04',
  '08',
  '17',
  '15',
  '15',
  '14',
  '02',
  '07',
  '02',
  '17',
  '17',
  '18',
  '20',
  '18',
  '08',
  '13',
  '15',
  '16',
  '16',
  '06',
  '19',
  '20',
  '19',
  '18',
  '19',
  '19',
  '16',
  '18',
  '15',
  '14',
  '18',
  '13',
  '15',
  '13',
  '11',
  '16',
  '11',
  '16',
  '09',
  '15',
  '18',
  '17',
  '17',
  '09',
  '20',
  '13',
  '17',
  '17',
  '20',
  '05',
  '19',
  '07',
  '20',
  '16',
  '16',
  '18',
  '15',
  '19',
  '08',
  '19',
  '19',
  '18',
  '19',
  '17',
  '18',
  '17',
  '16',
  '19',
  '19',
  '17',
  '16',
  '13',
  '18',
 